# Experimenting with various Kernel Combinations

## Rational Quadratic * Periodic

Here I will be experimenting with a range of different kernel combinations, to see which one is able to model sunspot number the best.

A list of kernel functions for PyMC3 can be found at https://docs.pymc.io/en/v3/api/gp/cov.html

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pymc3 as pm
import scipy as sp
import arviz as az
import theano.tensor as tt

In [2]:
df = pd.read_csv('../../Data/Savitzky-Golay Data/SVG_poly1.csv') # importing data for SVG polyorder = 1

In [3]:
# Define Sinusoidal Mean Function
class SinusoidalMean(pm.gp.mean.Mean):
    def __init__(self, amplitude):
        super().__init__()
        self.amplitude = amplitude
    
    def __call__(self, x):
        return tt.log(self.amplitude * tt.sin(x[:, 0] * np.pi / period + phase)**2 + offset)

## Data Preprocessing

In [4]:
limit = 65000 # training with the first 70k datapoints
skips = 280 # 1 data point per 10 luna cycle
train_data = df.iloc[:limit:skips] 
validation_data = df.iloc[limit::skips]

training_values = (train_data.Year.values).reshape(-1,1), train_data.Sunspot_Number.values
validation_values = (validation_data.Year.values).reshape(-1,1), validation_data.Sunspot_Number.values

training_data = training_values[0], np.log(training_values[1]) 
validation_data = validation_values[0], np.log(validation_values[1])

# take logs to obtain positive definite values
X_train = training_data[0]
y_train = training_data[1]

fig, ax = plt.subplots(2, dpi=125, figsize=[15,5])

ax[0].scatter(training_values[0], training_values[1], label='Training Values', s=1)
ax[0].scatter(validation_values[0], validation_values[1], label='Validation Values', s=1, color='tab:green')
ax[0].set_ylabel("# Sunspot")
ax[1].scatter(training_data[0], training_data[1], label='Training Data', s=1, color='red')
ax[1].scatter(validation_data[0], validation_data[1], label='Validation Data', s=1, color='tab:green')
ax[1].set_ylabel("Log # Sunspot")
ax[1].set_xlabel('Year')
fig.legend();

## GP Regression

In [5]:
# Define the Gaussian Process
with pm.Model() as model:
    
    # Define the phase of the sinusoidal mean
    phase = pm.Beta('phase', 2, 2)
    
    # Define the amplitude
    amplitude = pm.Gamma('amplitude', 7.5, 1)
    
    # Define the offset of the sinusoidal mean
    offset = pm.Gamma('offset', 2, 2)
    
    # Define the `Period' as a normal distribtion with mean = 11 years
    period = pm.Normal('Period', 11, 1)
    
    # Define the `length scale' of the periodic kernel
    length_scale = pm.Gamma('Length Scale 1', 7.5, 1)  
    
    # Define the 'length scale' of the RatQuad kernel
    length_scale2 = pm.Gamma('Length Scale 2', 7.5, 1)
    
    # Define 'alpha' of the rational-quadratic kernel
    alpha = pm.Gamma('alpha', 2, 2)
    
    # Define the `Standard deviation' as a beta function
    sigma = pm.Gamma('$\sigma$', 2, 2)
    
    # Define the periodic kernel
    cov_periodic = pm.gp.cov.Periodic(1, period=period, ls=length_scale)
    
    # Define the squared-exponential kernel
    cov_RatQuad = pm.gp.cov.RatQuad(1, alpha=alpha, ls=length_scale2)
    
    # Define the kernel
    cov = cov_RatQuad * cov_periodic
    
    sinusoidal_mean = SinusoidalMean(amplitude)
    
    gp = pm.gp.Marginal(cov_func=cov, mean_func=sinusoidal_mean)#sinusoidal_mean)  # implementation of the sum of a GP prior and noise

    y_ = gp.marginal_likelihood("y", X=X_train, y=y_train, noise=sigma)

    tr = pm.sample(return_inferencedata=True) # finds local maximimum 'a posteriori point' given a model

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [$\sigma$, alpha, Length Scale 2, Length Scale 1, Period, offset, amplitude, phase]


/Applications/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/Applications/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 4539 seconds.
There was 1 divergence after tuning. Increase `target_accept` or reparameterize.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.
The number of effective samples is smaller than 25% for some parameters.


In [6]:
az.plot_posterior(tr, linewidth=2, c='tab:blue', figsize=(21,3.2), textsize=12);
#plt.savefig('gp post.png'); 

In [7]:
X_new = validation_data[0]
with model:
    fnew = gp.conditional("f_taylor1", Xnew=X_new)

with model:
    ppc = pm.sample_posterior_predictive(tr, samples=1000, var_names=["f_taylor1"])#

/Applications/anaconda3/lib/python3.9/site-packages/pymc3/sampling.py:1689: UserWarning: samples parameter is smaller than nchains times ndraws, some draws and/or chains may not be represented in the returned posterior predictive sample
  warnings.warn(


In [8]:
from pymc3.gp.util import plot_gp_dist

# Sampling predictions
mu = np.exp(np.mean(ppc['f_taylor1'], axis=0))
std = np.exp(np.std(ppc['f_taylor1'], axis=0))

# draw plot
fig = plt.figure(figsize=(14, 5))
ax = fig.gca()
plot_gp_dist(ax, np.exp(ppc['f_taylor1']), X_new, palette="Blues")
# np.exp() to return log value to normal value

plt.scatter(X_new, mu, color = 'tab:green', label='Mean prediction')
# plot original data and GP
plt.plot(validation_data[0], np.exp(validation_data[1]), "or", ms=3, alpha=1.0, label="Validation Data")
plt.xlabel("Year")
plt.ylabel('Number of Sunspots')
plt.ylim(0,300)
plt.legend();
#plt.savefig('GD prediction');

In [9]:
observed = np.exp(validation_data[1])
predicted = mu

# Standard Deviation
std = np.std(observed - predicted) # std
normalised_std = np.array(std / np.mean(observed))

# Correlation
correlation = np.array(np.corrcoef(observed, predicted)[0, 1])

print(f'The normalised standard deviation is {normalised_std:.3f}, and the correlation is {correlation:.3f}.')

The normalised standard deviation is 0.496, and the correlation is 0.742.


In [10]:
''' Saving the Data '''
data = {"Metric": ["Standard Deviation (Normalised)", "Correlation Coefficient"], "Value": [normalised_std, correlation]}
df = pd.DataFrame(data)
#df.to_csv("../../Data/Taylor Diagram Data/GP_rational_periodic_taylor.csv", index=False)